Importing required files

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


64ng G859+

In [2]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
4.777949929000002
GPU (s):
0.1382895320000017
GPU speedup over CPU: 34x


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

upload the corpas

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 3000lines.txt to 3000lines.txt


# New section

Open and pre-process the data


In [ ]:
file = open("3000lines.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
 for i in lines:
  data = ' '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [ ]:
len(data)

230299

Apply tokenization and some other changes

In [ ]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  52767


array([[  12,   40,  227,    3],
       [  40,  227,    3,   13],
       [ 227,    3,   13,   23],
       [   3,   13,   23,   45],
       [  13,   23,   45,  657],
       [  23,   45,  657,   12],
       [  45,  657,   12,  658],
       [ 657,   12,  658,  859],
       [  12,  658,  859,    4],
       [ 658,  859,    4, 1749]])

In [ ]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)

In [ ]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 12  40 227]
 [ 40 227   3]
 [227   3  13]
 [  3  13  23]
 [ 13  23  45]
 [ 23  45 657]
 [ 45 657  12]
 [657  12 658]
 [ 12 658 859]
 [658 859   4]]
Response:  [   3   13   23   45  657   12  658  859    4 1749]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

 Creating the model

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add (Bidirectional(LSTM(256)))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 3, 10)             89890     
                                                                 
 bidirectional (Bidirection  (None, 512)               546816    
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 1000)              513000    
                                                                 
 dense_3 (Dense)             (None, 8989)              8997989   
                                                                 
Total params: 10147695 (38.71 MB)
Trainable params: 10147695 (38.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# from tensorflow import keras
# from keras.utils.vis_utils import plot_model

# keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

Train the model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='Accuracy', verbose=1, save_best_only=False )
model.compile(loss="categorical_crossentropy", metrics=['accuracy'],optimizer=Adam(learning_rate=0.001))
history = model.fit(X, y, validation_split= 0.2, epochs=60, batch_size=64, callbacks=[checkpoint])

Epoch 1/60
317/660 [=============>................] - ETA: 8s - loss: 7.4203 - accuracy: 0.0312

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['accuracy'],loc ='upper left')
plt.show

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['loss'],loc ='upper left')
plt.show

In [ ]:
y_hat = model.predict(X)

In [ ]:
y_pred = np.argmax(y_hat, axis=-1)

In [ ]:
len(y_pred)

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  print(predicted_word)
  return predicted_word

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

In [ ]:
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          text = text[:3]
          print(text)

          Predict_Next_Words(model, tokenizer, text)

      except Exception as e:
        print("Error occurred: ",e)
        continue